In [ ]:
from urllib.request import urlopen

import textract
import nltk

# MODIFY THESE LINES TO POINT TO YOUR WORD OR PDF FILE
str1 = '/Users/prajwalseth/Desktop/Ashoka/sem3/Mind and Behaviour/Week 1/Monday/'
str2 = "Rosen.pdf"


str = str1+str2
text = textract.process(str)

from readability.readability import Document
from bs4 import BeautifulSoup

readable_article = Document(text).summary()
readable_title = Document(text).title()
soup = BeautifulSoup((text), "lxml")

tokens = [word for sent in nltk.sent_tokenize(soup.text) for word in nltk.word_tokenize(sent)]

from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
stemmed_tokens = [stemmer.stem(t) for t in tokens]

import datetime, re, sys
from sklearn.feature_extraction.text import TfidfVectorizer

def tokenize_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

token_dict = {}

for article in text:
    token_dict[article] = soup.text
        
tfidf = TfidfVectorizer(tokenizer=tokenize_and_stem, stop_words='english', decode_error='ignore')
sys.stdout.flush()

tdm = tfidf.fit_transform(token_dict.values()) # this can take some time (about 60 seconds on my machine)

from random import randint

feature_names = tfidf.get_feature_names()

import math
from __future__ import division
import collections

article_id = 0
#article_text = reuters.raw(reuters.fileids()[article_id])

article_text = soup.text
sent_scores = []
for sentence in nltk.sent_tokenize(article_text):
    
    score = 0
    sent_tokens = tokenize_and_stem(sentence)
    for token in (t for t in sent_tokens if t in feature_names):
        score += tdm[article_id, feature_names.index(token)]
    
    if len(sent_tokens) ==0:
        continue;
    
    else:
        sent_scores.append((score / len(sent_tokens), sentence))

summary_length = int(math.ceil(len(sent_scores) / (3)))
var = 0
order = {}
sent_scores.sort(key=lambda sent: sent[0], reverse=True)
print ('*** SUMMARY ***')
for summary_sentence in sent_scores[:summary_length]:
    var = article_text.find(summary_sentence[1])
    order[var] = summary_sentence[1]
    # print (summary_sentence[1])
ordered = collections.OrderedDict(sorted(order.items()))
for k, v in ordered.items(): 
    print(v) #end = " ")
print('\n')
print ('\n*** ORIGINAL ***')
print (article_text)